# Ejercicio 9: Uso de la API de OpenAI

En este ejercicio vamos a aprender a utilizar la API de OpenAI

## 1. Uso básico

El siguiente código sirve para conectarse con la API de OpenAI de forma básica

In [1]:
from openai import OpenAI

In [2]:
OPENAI_API_KEY = ''

In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [48]:
response = client.responses.create(
    model="gpt-4.1",
    input="¿Quién descubrió América?"
)

print(response.output_text)

América fue "descubierta" para Europa por **Cristóbal Colón**, un navegante genovés al servicio de los Reyes Católicos de España. Colón llegó al continente americano el **12 de octubre de 1492**, arribando a lo que hoy conocemos como el Caribe (las islas Bahamas, probablemente la isla Guanahaní, llamada San Salvador por él).

Es importante aclarar que antes de la llegada de Colón, América ya estaba habitada por diversos pueblos indígenas desde hacía miles de años y, además, exploradores vikingos como Leif Erikson habían llegado a una región en el actual Canadá (Vinland) aproximadamente en el año 1000, mucho antes que Colón, aunque ese "descubrimiento" no tuvo un impacto global en Europa.

Resumen:
- **Cristóbal Colón** es quien usualmente se le atribuye el "descubrimiento de América" desde la perspectiva europea, en 1492. 
- Sin embargo, **América ya estaba habitada** y había sido visitada antes por otros pueblos, como los vikingos.


## 2. Retrieval

### 2.1 Cargo el corpus de 20 News Groups

In [8]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [15]:
import pandas as pd
import pymupdf

path = ''
file = "irbookonlinereading.pdf"

# Abrir el PDF
doc = pymupdf.open(path + file)

In [21]:
# Extraer texto por página
data = []
for i, page in enumerate(doc):
    text = page.get_text().encode("utf8")
    if text.strip():  # ignorar páginas en blanco
        data.append({
            "page": i + 1,
            "raw": str(text).strip()
        })

# Convertir a DataFrame
df = pd.DataFrame(data)
df

,page,raw
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
1,2,b'Online edition (c)\n2009 Cambridge UP\n'
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...


### 2.2 Transformo a embeddings

In [22]:
from sentence_transformers import SentenceTransformer

In [23]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [24]:
embeddings = model.encode(df['raw'])

In [25]:
df['embeddings'] = embeddings.tolist()
df

,page,raw,embeddings
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,"[-0.06956461817026138, -0.09505553543567657, 0..."
1,2,b'Online edition (c)\n2009 Cambridge UP\n',"[-0.049505531787872314, -0.08177918940782547, ..."
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,"[-0.04497804492712021, -0.12156280130147934, 0..."
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,"[-0.08138851076364517, -0.031383614987134933, ..."
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,"[-0.05637805536389351, -0.08665604889392853, -..."
...,...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...,"[-0.07100450992584229, -0.09812875092029572, -..."
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...,"[0.005868136882781982, -0.11765123903751373, -..."
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...,"[-0.08848388493061066, -0.0766187533736229, -0..."
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...,"[-0.05640826374292374, -0.11733546853065491, -..."


### 2.3 Creo una query y hago la búsqueda

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
query = "¿Quién descubrió América?"
query_emb = model.encode(query)
print(query_emb)

[ 1.32415788e-02 -1.38535984e-02 -4.86043058e-02 -3.16161029e-02
 -3.19868959e-02 -3.69648039e-02  5.69050834e-02 -1.62796881e-02
 -2.64756382e-02  5.96109312e-03  5.33242524e-02 -3.47383730e-02
 -2.65685711e-02  2.55968347e-02 -1.81286428e-02 -7.60501204e-03
 -3.20969336e-02 -3.68653201e-02 -2.04010797e-03  2.25827452e-02
  6.93910345e-02 -7.77236884e-03 -4.22271714e-02  1.10106841e-01
  1.33998496e-02 -3.18524875e-02  7.74564743e-02  3.63411307e-02
 -3.05152051e-02 -2.70387437e-02 -9.34987236e-03  1.33743575e-02
  6.63040429e-02  4.17424068e-02  3.16561460e-02 -1.99810546e-02
  6.29395768e-02 -1.07368931e-01 -3.68436016e-02  4.76068631e-03
 -1.26932293e-01  3.25190078e-04  3.53902727e-02  2.19949409e-02
  1.04914175e-03 -3.12276091e-02 -5.24926521e-02  1.80567771e-01
  3.61488722e-02 -1.08434865e-02  5.39276376e-02  1.40397716e-02
  2.37098597e-02  1.52820256e-02  5.85652702e-02  1.85835734e-02
  2.10681204e-02 -4.72157709e-02  4.02341150e-02  4.79112118e-02
 -6.75087199e-02 -5.10276

In [42]:
import numpy as np

embeddings_array = np.vstack(df['embeddings'].values)
similarities = cosine_similarity(query_emb.reshape(1, -1), embeddings_array)

Obtengo los 5 documentos más similares a mi query

In [43]:
top_k = 5
top_indices = similarities[0].argsort()[-top_k:][::-1]
df.iloc[top_indices][['page', 'raw']]

,page,raw
415,416,b'Online edition (c)\n2009 Cambridge UP\n17.1\...
515,516,b'Online edition (c)\n2009 Cambridge UP\n21.3\...
433,434,b'Online edition (c)\n2009 Cambridge UP\n17.7\...
571,572,b'Online edition (c)\n2009 Cambridge UP\nAutho...
419,420,b'Online edition (c)\n2009 Cambridge UP\n17.2\...


In [44]:
context = "\n\n".join(df.iloc[top_indices]['raw'].values)

In [45]:
context

"b'Online edition (c)\\n2009 Cambridge UP\\n17.1\\nHierarchical agglomerative clustering\\n379\\n1.0\\n0.8\\n0.6\\n0.4\\n0.2\\n0.0\\nAg trade reform.\\nBack\\xe2\\x88\\x92to\\xe2\\x88\\x92school spending is up\\nLloyd\\xe2\\x80\\x99s CEO questioned\\nLloyd\\xe2\\x80\\x99s chief / U.S. grilling\\nViag stays positive\\nChrysler / Latin America\\nOhio Blue Cross\\nJapanese prime minister / Mexico\\nCompuServe reports loss\\nSprint / Internet access service\\nPlanet Hollywood\\nTrocadero: tripling of revenues\\nGerman unions split\\nWar hero Colin Powell\\nWar hero Colin Powell\\nOil prices slip\\nChains may raise prices\\nClinton signs law\\nLawsuit against tobacco companies\\nsuits against tobacco firms\\nIndiana tobacco lawsuit\\nMost active stocks\\nMexican markets\\nHog prices tumble\\nNYSE closing averages\\nBritish FTSE index\\nFed holds interest rates steady\\nFed to keep interest rates steady\\nFed keeps interest rates steady\\nFed keeps interest rates steady\\n\\xe2\\x97\\xaeFigu

In [46]:
prompt = f"""Eres una aplicación de Retrieval Augmented Generation que siempre responde en español. Usa el siguiente contexto para responder la pregunta. 
Si la respuesta no está en el contexto, di que no sabes.

Contexto:
{context}

Pregunta:
El usuario está preguntando sobre: {query}
"""

In [47]:
response = client.responses.create(
    model="gpt-4.1",
    temperature=0.3,
    input=prompt
)

print(response.output_text)

No lo sé.
